# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import re
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [2]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Oluga\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Oluga\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# load data from database

def load_data():
    engine = create_engine('sqlite:///DisasterResponse.db')
    sql = 'SELECT * from messages_and_categories;'
    df = pd.read_sql(sql, engine)
    
    X = df[['message']]
    y = df[['related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report']]

    return X, y

In [4]:
X, y = load_data()
X.sample(5)

,message
25481,These remaining cases are due more to endemic ...
25827,Overland corridors: WFP has several logistical...
2940,"Please, if it rains, what would I do?"
17534,"As of Wednesday, a total of 59,337 fishermen a..."
5375,PAM tell everyone the color of the cards (vouc...


In [5]:
y.sample(5)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
4025,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,1,0,1
22622,1,0,0,1,0,0,0,0,0,0,...,1,1,1,1,1,0,1,0,1,0
23969,1,0,0,1,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
12323,1,1,0,1,0,0,0,0,0,0,...,0,1,1,1,0,0,0,0,0,1
4742,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


### 2. Write a tokenization function to process your text data

In [6]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = [lemmatizer.lemmatize(token).lower().strip() for token in tokens]

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
def build_model():
    pipeline = Pipeline([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),
        ('knn_clf', KNeighborsClassifier()),
        #('clf', MultiOutputClassifier('knn_clf'))
    ])

    knn_params = {
        'knn_clf__algorithm': ['ball_tree', 'kd_tree', 'brute'],
        'knn_clf__n_neighbors': [3, 5, 11, 19],
        'knn_clf__weights': ['uniform', 'distance'],
        'knn_clf__metric': ['euclidian', 'manhattan']
    }

    cv = GridSearchCV(pipeline, param_grid=knn_params)

    return cv

In [8]:
pipeline = Pipeline([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),
        ('knn_clf', KNeighborsClassifier()) #,
        #('clf', MultiOutputClassifier('knn_clf'))
    ])
sorted(pipeline.get_params().keys())

['knn_clf',
 'knn_clf__algorithm',
 'knn_clf__leaf_size',
 'knn_clf__metric',
 'knn_clf__metric_params',
 'knn_clf__n_jobs',
 'knn_clf__n_neighbors',
 'knn_clf__p',
 'knn_clf__weights',
 'memory',
 'steps',
 'text_pipeline',
 'text_pipeline__memory',
 'text_pipeline__steps',
 'text_pipeline__tfidf',
 'text_pipeline__tfidf__norm',
 'text_pipeline__tfidf__smooth_idf',
 'text_pipeline__tfidf__sublinear_tf',
 'text_pipeline__tfidf__use_idf',
 'text_pipeline__vect',
 'text_pipeline__vect__analyzer',
 'text_pipeline__vect__binary',
 'text_pipeline__vect__decode_error',
 'text_pipeline__vect__dtype',
 'text_pipeline__vect__encoding',
 'text_pipeline__vect__input',
 'text_pipeline__vect__lowercase',
 'text_pipeline__vect__max_df',
 'text_pipeline__vect__max_features',
 'text_pipeline__vect__min_df',
 'text_pipeline__vect__ngram_range',
 'text_pipeline__vect__preprocessor',
 'text_pipeline__vect__stop_words',
 'text_pipeline__vect__strip_accents',
 'text_pipeline__vect__token_pattern',
 'text_p

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
def display_results(cv, y_test, y_pred):
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)
    print("\nBest Parameters:", cv.best_params_)

In [10]:
def main():
    X, y = load_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    model = build_model()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    display_results(model, y_test, y_pred)

main()

c:\tutes\udacity\disaster_response_pipeline_project\venv_disaster_response\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
240 fits failed out of a total of 240.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
96 fits failed with the following error:
Traceback (most recent call last):
  File "c:\tutes\udacity\disaster_response_pipeline_project\venv_disaster_response\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\tutes\udacity\disaster_response_pipeline_project\venv_disaster_response\lib\site-packages\sklearn\pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File 

ValueError: Found input variables with inconsistent numbers of samples: [1, 19662]

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
display_results(cv, y_test, y_pred)

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.